# Before

In [ ]:
from IPython.display import clear_output
!pip install autogluon --user
clear_output()

import numpy as np
import pandas as pd 
import os 
import random

from sklearn.preprocessing import StandardScaler, MinMaxScaler,MaxAbsScaler,RobustScaler
from autogluon.tabular import TabularDataset, TabularPredictor

TRAIN_PATH = "../input/spaceship-titanic/train.csv"
TEST_PATH = "../input/spaceship-titanic/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/spaceship-titanic/sample_submission.csv"
SUMISSION_PATH = "submission.csv"

ID = "PassengerId"
TARGET = 'Transported'
EVAL_METRIC = "accuracy"


SAVE_PATH = 'agModels-predictClass' 
DEFAULT_RANDOM_SEED = 2022


def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seedBasic()

train = TabularDataset(TRAIN_PATH)
test = TabularDataset(TEST_PATH)

In [ ]:
total = pd.concat([train,test],axis=0)

num_col = []
for col in train:
    if train[col].dtypes != "object" and col != TARGET:
        num_col.append(col)
        
scaler = StandardScaler()
total[num_col] = scaler.fit_transform(total[num_col])

train_num = len(train)

train = total[:train_num]
test = total[train_num:]
test = test.drop([TARGET],axis=1)

In [ ]:
feature = [col for col in train.columns if col != TARGET and col != ID]
train = train[train[feature].duplicated()==False]

# Build

In [ ]:
predictor = TabularPredictor(
    label=TARGET,
    eval_metric=EVAL_METRIC, 
    path=SAVE_PATH).fit(train)

# After

In [ ]:
y_pred = predictor.predict(test)

submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] = y_pred
submission.to_csv(SUMISSION_PATH,index=False)
submission.head()